# Pandas(iii)

In this notebook we will continue using our previous sales dataset `sales-dataset-2.csv`

In [ ]:
# Load pandas and the df here
import pandas as pd
store_df = pd.read_csv("https://drive.google.com/u/1/uc?id=1kbCDXONvf8Bn4kEW1ssmxdJLDHC-oV-E&export=download")
store_df.head()

## 1. Sorting

### 1.1 .sort_values()

We can reorder the rows (in a df) or values(in a series)

```python
df.sort_values('column_name')
df.sort_values(['column_name1', 'column_name2'])

sr.sort_values()
```

In [ ]:
# Demonstration
store_df.sort_values('Date').head()

**Exercise**

By checking the documentation [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html), perform the following tasks:

1. Sort by ascending order of `Store` and `Dept` and descending order of `Date`
2. What is the data type of `Date`? You can check by inspecting the first element in the `Date` column. Hence, explain why the sorting in question 1 does not work.
3. Create a new column `Date_dt`, with the values are datetime instead of string.  Then run the sorting again. (Hint: You will need to use `.strptime()` from datetime package and `.apply()` of pandas series)

In [ ]:
# Type your code here


### 1.2 .sort_index()

It is also possible to sort by the index.

In [ ]:
store_df.sort_index(ascending=False)

## 2. Aggregate functions

There are a lot of aggregation functions.  Here's some of the common ones:
```python
    series.count()
    series.sum()
    series.mean()
    series.max()
    series.min()
    series.unique()
    series.nunique()
```
Try discovering the rest of them.

**Example 1**

In [ ]:
# Demonstration
df1 = pd.DataFrame({
    'Courses':["Spark", "PySpark", "Hadoop", "Python", "PySpark", "Spark"],
    'Fee' :[20000, 25000, 26000, 22000, 24000, 35000],
    'Duration':[30, 40, 35, 40, 60, 60],
    'Discount':[1000, 2300, 1200, 2500, 2000, 2000]
})
df1

In [ ]:
# Number of unique courses
df1['Courses'].nunique()

In [ ]:
# Average course duration
df1['Duration'].mean()

**Example 2**

This is the dataframe we encountered in the first pandas lesson.

In this example we calculate the row-sum and column-sum of the dataframe.

In [ ]:
import pandas as pd
clinic_df = pd.DataFrame([
        ['January', 100, 100, 23, 100],
        ['February', 51, 45, 145, 45],
        ['March', 81, 96, 65, 96],
        ['April', 80, 80, 54, 180],
        ['May', 51, 54, 54, 154],
        ['June', 112, 109, 79, 129]
    ],
    columns=['month', 'clinic_east','clinic_north', 'clinic_south','clinic_west']
)

with_row_sum = clinic_df.assign(row_sum=clinic_df.sum(axis=1, numeric_only=True))

with_row_col_sum = pd.concat([with_row_sum, with_row_sum.sum(axis=0).to_frame().T])

with_row_col_sum

**Exercise**

> Try to create some summary of the dataset `store_df`:
1. What is the average weekly sales?
2. How many difference stores are there?
3. Which weeks are holidays?

In [ ]:
# Type your code below


## 3. Group by

### 3.1 Groupby - aggregate pattern

Group by is an essential feature in every table manipulation. In pandas, we use the function
```python
# Group by a particular column
df.groupby("Column name")

```
After a group by, we must provide further aggregation instruction so that we can obtain usable information.
```python
# Follow the groupby by a sum()
df.groupby("Column name").sum()

# Follow the groupby by a max()
df.groupby("Column name").max()

# Follow the groupby by your own function
df.groupby("Column name").agg(func)

# Follow the groupby by your multiple functions
df.groupby("Column name").agg([func1, func2])

# Follow the groupby by different functions for different columns
df.groupby("Column name").agg({"col1": func1, "col2": func2})
```

In [ ]:
# Demonstaration
df2 = pd.DataFrame({'Animal': ['Falcon', 'Falcon',
                              'Parrot', 'Parrot'],
                   'Max Speed': [380., 370., 24., 26.]})
df2

In [ ]:
# Group by, followed by aggregation
df2.groupby('Animal').mean()

In [ ]:
# Group by, followed by aggregation
df2.groupby('Animal').agg(['sum','mean'])

**Exercise**

1. get the sum of sales for every store
2. find the best performing store

In [ ]:
# Type your code here


### 3.2 The GroupBy object

After a `.groupby()` function is run, a `DataFrameGroupBy` object is produced.

A `DataFrameGroupBy` object represents a intermediate result where records are grouped but not aggregated yet.

In this stage, we may further select the columns before proceed to aggregation.

- `[['col1', 'col2]]` returns a `DataFrameGroupBy`

- `['col']` syntax returns a `SeriesGroupBy` object

In the following example, you can expect the result to be a `DataFrameGroupBy` object

In [ ]:
# Demonstration
store_df.groupby(['Store','Dept'])[['Weekly_Sales', 'Date']]

### 3.3 The special `count()` aggregation

Count is different from other aggregation method like `sum` and `max` because we do not need numbers to be calculated.

In [ ]:
# Demonstration

df1 = pd.DataFrame({
    'Courses':["Spark", "PySpark", "Hadoop", "Python", "PySpark", "Spark"],
    'Fee' :[20000, 25000, 26000, 22000, 24000, 35000],
    'Duration':[30, 40, 35, 40, 60, 60],
    'Discount':[1000, 2300, 1200, 2500, 2000, 2000]
})
df1

In the above dataframe, to check the number of each course:
- Spark: 2
- PySpark: 2
- Hadoop: 1
- Python: 1

The standard group by syntax will be:

In [ ]:
df1.groupby('Courses').count()

However, the result is weird since the `count()` aggregation method is applied on all the three columns `Fee`, `Duration` and `Discount`.

Turns out the correct way is to use `.value_counts`.

In [ ]:
# Demonstration

df1['Courses'].value_counts()

### 3.4 Time series Group By

What is time series?

Time series is a series that represent measures in regular time interval. Examples include:

- Per minute sleep quality
- Hourly temperature
- Daily sales volume


#### 3.4.1 .resample()

While modeling time series can be very valuable, we must first preprocess them such that the interval between records are fixed.

`.resample()` is a way to group by a time interval (e.g. 1 day)

**Example**

In the following dataframe, temperatures are recorded but they have irregular time intervals.

In [ ]:
temperature_record = pd.DataFrame([
    ["1988-04-07 00:00", 30],
    ["1988-04-07 12:00", 29],
    ["1988-04-08 00:00", 30],
    ["1988-04-09 00:00", 29],
    ["1988-04-09 12:00", 28],
    ["1988-04-10 03:00", 31],
], columns=['time', 'reading'])

# to_datetime() is a short cut of .apply() + .strptime()
temperature_record.index = pd.to_datetime(temperature_record['time'], format="%Y-%m-%d %H:%M")

# Now the index is a DateTimeIndex - A special one for datetime

# Then use .sample() as a group-by
resampled = temperature_record.resample("1d")['reading'].mean()

resampled

#### 3.4.2 .rolling()

The analytic methods "moving average" or "rolling sum" are achieved with the `.rolling()` method.

**Example**

We wish to generate a 2-days moving average on the previous dataframe.

Note: this method does not require the index to be a datetime index.

In [ ]:
resampled.rolling(2).mean()

**Checkpoint**

Why do we apply rolling to `resample` but not to `temperature_record`?
